<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# Model Classes

The model classes represent the fundamental building blocks to model a financial market. They are used to represent the fundamental risk factors driving uncertainty (e.g. a stock, an equity index an interest rate). The following models are available:

* `geometric_brownian_motion`: Black-Scholes-Merton (1973) geometric Brownian motion
* `jump_diffusion`: Merton (1976) jump diffusion
* `stochastic_volatility`: Heston (1993) stochastic volatility model
* `stoch_vol_jump_diffusion`: Bates (1996) stochastic volatility jump diffusion
* `sabr_stochastic_volatility`: Hagan et al. (2002) stochastic volatility model
* `mean_reverting_diffusion`: Vasicek (1977) short rate model
* `square_root_diffusion`: Cox-Ingersoll-Ross (1985) square-root diffusion
* `square_root_jump_diffusion`: square-root jump diffusion (experimental)
* `square_root_jump_diffusion_plus`: square-root jump diffusion plus term structure (experimental)

In [1774]:
from dx import *
from pylab import plt
plt.style.use('seaborn')

In [1775]:
np.set_printoptions(precision=3)

Throughout this section we fix a `constant_short_rate` discounting object.

In [1776]:
r = constant_short_rate('r', 0.01)

## geometric_brownian_motion

To instantiate any kind of model class, you need a `market_environment` object conataining a minimum set of data (depending on the specific model class).

In [1777]:
me = market_environment(name='me', pricing_date=dt.datetime(2021, 1, 1))

For the **geometric Browniam motion class**, the minimum set is as follows with regard to the constant parameter values. Here, we simply make assumptions, in practice the single values would, for example be retrieved from a data service provider like Thomson Reuters or Bloomberg. The frequency parameter is according to the pandas frequency conventions (cf. <a href="http://pandas.pydata.org/pandas-docs/stable/timeseries.html" target="_blank">http://pandas.pydata.org/pandas-docs/stable/timeseries.html</a>).

In [1778]:
me.add_constant('initial_value', 100)
me.add_constant('volatility', 0.2)
me.add_constant('final_date', dt.datetime(2021, 12, 31))
# time horizon for the simulation
me.add_constant('currency', 'EUR')
me.add_constant('frequency', 'M')
# monthly frequency; paramter accorind to pandas convention
number_of_paths = 2
me.add_constant('paths',   number_of_paths)
# number of paths for simulation

Every model class needs a discounting object since this defines the risk-neutral drift of the risk factor.

In [1779]:
me.add_curve('discount_curve', r)

The instantiation of a model class is then accomplished by providing a name as a `string` object and the respective `market_environment` object.

In [1780]:
gbm = geometric_brownian_motion('gbm', me)

The `generate_time_grid` method generates a `ndarray` objet of `datetime` objects given the specifications in the market environment. This represents the discretization of the time interval between the `pricing_date` and the `final_date`. This method does not need to be called actively.

In [1781]:
gbm.generate_time_grid()

In [1782]:
gbm.time_grid

array([datetime.datetime(2021, 1, 1, 0, 0),
       datetime.datetime(2021, 1, 31, 0, 0),
       datetime.datetime(2021, 2, 28, 0, 0),
       datetime.datetime(2021, 3, 31, 0, 0),
       datetime.datetime(2021, 4, 30, 0, 0),
       datetime.datetime(2021, 5, 31, 0, 0),
       datetime.datetime(2021, 6, 30, 0, 0),
       datetime.datetime(2021, 7, 31, 0, 0),
       datetime.datetime(2021, 8, 31, 0, 0),
       datetime.datetime(2021, 9, 30, 0, 0),
       datetime.datetime(2021, 10, 31, 0, 0),
       datetime.datetime(2021, 11, 30, 0, 0),
       datetime.datetime(2021, 12, 31, 0, 0)], dtype=object)

The simulation itself is initiated by a call of the method `get_instrument_values`. It returns an `ndarray` object containing the simulated paths for the risk factor.

In [1783]:
paths = gbm.get_instrument_values()

In [1784]:
paths[:, :2]

array([[100.   , 100.   ],
       [103.008,  96.921],
       [102.689,  97.073],
       [109.181,  91.145],
       [106.021,  93.708],
       [109.987,  90.175],
       [108.782,  91.025],
       [103.772,  95.258],
       [109.809,  89.867],
       [104.986,  93.842],
       [111.871,  87.916],
       [103.547,  94.828],
       [ 92.185, 106.336]])

These can, for instance, be visualized easily. First some plotting parameter specifications we want to use throughout.

In [1785]:
%matplotlib inline
colormap='RdYlBu_r'
lw=1.25
figsize=(10, 6)
legend=False
no_paths=10

For easy plotting, we put the data with the `time_grid` information into a pandas `DataFrame` object.

In [1786]:
pdf = pd.DataFrame(paths, index=gbm.time_grid)

The following visualizes the first 10 paths of the simulation.

In [ ]:
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

## jump_diffusion

The next model is the **jump diffusion model from Merton (1976)** adding a log-normally distributed jump component to the geometric Brownian motion. Three more parameter values are needed:

In [ ]:
me.add_constant('lambda', 0.7)
  # probability for a jump p.a.
me.add_constant('mu', -0.8)
  # expected relative jump size
me.add_constant('delta', 0.1)
  # standard deviation of relative jump

The instantiation of the model class and usage then is the same as before.

In [ ]:
jd = jump_diffusion('jd', me)

In [ ]:
paths = jd.get_instrument_values()

In [ ]:
pdf = pd.DataFrame(paths, index=jd.time_grid)

In [ ]:
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

## stochastic_volatility

Another important financial model is the  **stochastic volatility model according to Heston (1993)**. Compared to the geometric Brownian motion, this model class need four more parameter values.

In [ ]:
me.add_constant('rho', -.7)
  # correlation between risk factor process (eg index)
  # and variance process 
me.add_constant('kappa', 2.5)
  # mean reversion factor
me.add_constant('theta', 0.1)
  # long-term variance level
me.add_constant('vol_vol', 0.1)
  # volatility factor for variance process

Again, the instantiation and usage of this model class are essentially the same.

In [ ]:
sv = stochastic_volatility('sv', me)

The following visualizes 10 simulated paths for the **risk factor process**.

In [ ]:
paths = sv.get_instrument_values()

In [ ]:
pdf = pd.DataFrame(paths, index=sv.time_grid)

In [ ]:
# index level paths
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

This model class has a second set of simulated paths, namely for the **variance process**.

In [ ]:
vols = sv.get_volatility_values()

In [ ]:
pdf = pd.DataFrame(vols, index=sv.time_grid)

In [ ]:
# volatility paths
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

## stoch_vol_jump_diffusion

The next model class, i.e. `stoch_vol_jump_diffusion`, combines **stochastic volatility with a jump diffusion according to Bates (1996).** Our market environment object `me` contains already all parameters needed for the instantiation of this model.

In [ ]:
svjd = stoch_vol_jump_diffusion('svjd', me)

As with the `stochastic_volatility` class, this class generates simulated paths for both the **risk factor and variance process**.

In [ ]:
paths = svjd.get_instrument_values()

In [ ]:
pdf = pd.DataFrame(paths, index=svjd.time_grid)

#print of values
print (pdf.to_string())
barrier = 70.0
defaultingPaths = 0
numeroColonne = 0
numeroColonna = 0
numeroRiga = 0
for row in pdf.iterrows():
    numeroColonne = len(pdf.columns)

    numeroColonna = 0
    while numeroColonna < numeroColonne:
        print ('numeroriga = ' + str(numeroRiga) + 'numerocolonna=' + str(numeroColonna))


        miaCella = pdf.iloc[numeroRiga, numeroColonna]
        mioValore = float(miaCella)

        print ('cella=' + str(mioValore))

        if mioValore < barrier:
            defaultingPaths = defaultingPaths + 1
            print ('defaulting path for column' + str(numeroColonna) + ' and row '+  str(numeroRiga) + ' and value ' +  + str(mioValore))
        numeroColonna = numeroColonna + 1
    numeroRiga = numeroRiga + 1
print (defaultingPaths)


In [ ]:
# index level paths
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

In [ ]:
vols = svjd.get_volatility_values()

In [ ]:
pdf = pd.DataFrame(vols, index=svjd.time_grid)

In [ ]:
# volatility paths
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

## sabr_stochastic_volatility

The `sabr_stochastic_volatility` model is based on the **Hagan et al. (2002)** paper. It needs the following parameters:

In [ ]:
# short rate like parameters
me.add_constant('initial_value', 0.5)
  # starting value (eg inital short rate)
me.add_constant('alpha', 0.04)
  # initial variance
me.add_constant('beta', 0.5)
  # exponent
me.add_constant('rho', 0.1)
  # correlation factor
me.add_constant('vol_vol', 0.5)
  # volatility of volatility/variance

In [ ]:
sabr = sabr_stochastic_volatility('sabr', me)

In [ ]:
paths = sabr.get_instrument_values()

In [ ]:
pdf = pd.DataFrame(paths, index=sabr.time_grid)

In [ ]:
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

In [ ]:
vols = sabr.get_volatility_values()

In [ ]:
pdf = pd.DataFrame(vols, index=sabr.time_grid)

In [ ]:
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

## mean_reverting_diffusion

The `mean_reverting_diffusion` model class is based on the **Vasicek (1977)** short rate model. It is suited to model mean-reverting quantities, like short rates, volatilities, etc. The model needs the following set of parameters:

In [ ]:
# short rate like parameters
me.add_constant('initial_value', 0.05)
  # starting value (eg inital short rate)
me.add_constant('volatility', 0.05)
  # volatility factor
me.add_constant('kappa', 2.5)
  # mean reversion factor
me.add_constant('theta', 0.01)
  # long-term mean

In [ ]:
mrd = mean_reverting_diffusion('mrd', me)

In [ ]:
paths = mrd.get_instrument_values()

In [ ]:
pdf_1 = pd.DataFrame(paths, index=mrd.time_grid)

Simulated paths can go negative which might be undesireable, e.g. when modeling volatility processes.

In [ ]:
pdf_1[pdf_1.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

Using a full trunctation discretization scheme ensures positive paths.

In [ ]:
mrd = mean_reverting_diffusion('mrd', me, truncation=True)
paths = mrd.get_instrument_values()
pdf_1 = pd.DataFrame(paths, index=mrd.time_grid)

In [ ]:
pdf_1[pdf_1.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

## square_root_diffusion

The `square_root_diffusion` model class is based on the **square-root diffusion according to Cox-Ingersoll-Ross (1985)**. This class is often used to model stochastic short rates or a volatility process (eg like the VSTOXX volatility index). The model needs the same parameters as the `mean_reverting_diffusion`.

As before, the handling of the model class is the same, making it easy to simulate paths given the parameter specifications and visualize them.

In [ ]:
srd = square_root_diffusion('srd', me)

In [ ]:
paths = srd.get_instrument_values()

In [ ]:
pdf_2 = pd.DataFrame(paths, index=srd.time_grid)

In [ ]:
pdf_2[pdf_2.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

Let us compare the simulated paths --- based on the same parameters --- for the `mean_reverting_diffusion` and the `square_root_diffusion` in a single chart. The paths from the `mean_reverting_diffusion` object are much more volatile.

In [ ]:
ax = pdf_1[pdf_1.columns[:no_paths]].plot(style='b-.', lw=lw,
                           figsize=figsize, legend=legend);
pdf_2[pdf_1.columns[:no_paths]].plot(style='r--', lw=lw,
                           figsize=figsize, legend=legend, ax=ax);

However, the *mean values* do not differ that much since both share the same long-term mean level. Mean reversion is faster with the `square_root_diffusion` model.

In [ ]:
ax = pdf_1.mean(axis=1).plot(style='b-.', lw=lw,
                    figsize=figsize, label='mrd', legend=True);
pdf_2.mean(axis=1).plot(style='r--', lw=lw,
                    figsize=figsize, label='srd', legend=True);

## square_root_jump_diffusion

*Experimental Status*

Building on the square-root diffusion, there is a **square-root jump diffusion** adding a log-normally distributed jump component. The following parmeters might be for a volatility index, for example. In this case, the major risk might be a large positive jump in the index. The following model parameters are needed:

In [ ]:
# volatility index like parameters
me.add_constant('initial_value', 25.)
  # starting values
me.add_constant('kappa', 2.)
  # mean-reversion factor
me.add_constant('theta', 20.)
  # long-term mean
me.add_constant('volatility', 1.)
  # volatility of diffusion
me.add_constant('lambda', 0.3)
  # probability for jump p.a.
me.add_constant('mu', 0.4)
  # expected jump size
me.add_constant('delta', 0.2)
  # standard deviation of jump

Once the `square_root_jump_diffusion` class is instantiated, the handling of the resulting object is the same as with the other model classes.

In [ ]:
srjd = square_root_jump_diffusion('srjd', me)

In [ ]:
paths = srjd.get_instrument_values()

In [ ]:
pdf = pd.DataFrame(paths, index=srjd.time_grid)

In [ ]:
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

In [ ]:
paths[-1].mean()

## square_root_jump_diffusion_plus

*Experimental Status*

This model class further enhances the `square_root_jump_diffusion` class by adding a **deterministic shift approach according to Brigo-Mercurio (2001)** to account for a market given term structure (e.g. in volatility, interest rates). Let us define a simple term structure as follows:

In [ ]:
term_structure = np.array([(dt.datetime(2015, 1, 1), 25.),
                  (dt.datetime(2015, 3, 31), 24.),
                  (dt.datetime(2015, 6, 30), 27.),
                  (dt.datetime(2015, 9, 30), 28.),
                  (dt.datetime(2015, 12, 31), 30.)])

In [ ]:
me.add_curve('term_structure', term_structure)

In [ ]:
srjdp = square_root_jump_diffusion_plus('srjdp', me)

The method `generate_shift_base` calibrates the square-root diffusion to the given term structure (varying the parameters `kappa`, `theta` and `volatility`). 

In [ ]:
srjdp.generate_shift_base((2.0, 20., 0.1))

The results are `shift_base` values, i.e. the difference between the model and market implied foward rates.

In [ ]:
srjdp.shift_base
  # difference between market and model
  # forward rates after calibration

The method `update_shift_values` then calculates deterministic shift values for the relevant time grid by interpolation of the `shift_base` values.

In [ ]:
srjdp.update_shift_values()

In [ ]:
srjdp.shift_values
  # shift values to apply to simulation scheme
  # given the shift base values

When simulating the process, the model forward rates ...

In [ ]:
srjdp.update_forward_rates()
srjdp.forward_rates
  # model forward rates resulting from parameters

... are then shifted by the `shift_values` to better match the term structure.

In [ ]:
srjdp.forward_rates[:, 1] + srjdp.shift_values[:, 1]
  # shifted foward rates

The simulated paths then are including the deterministic shift.

In [ ]:
paths = srjdp.get_instrument_values()

In [ ]:
pdf = pd.DataFrame(paths, index=srjdp.time_grid)

In [ ]:
pdf[pdf.columns[:no_paths]].plot(colormap=colormap, lw=lw,
                           figsize=figsize, legend=legend);

The effect might not be immediately visible in the paths plot, however, the mean of the simulated values in this case is higher by about 1 point compared to the `square_root_jump_diffusion` simulation without deterministic shift.

In [ ]:
paths[-1].mean()

**Copyright, License & Disclaimer**

© Dr. Yves J. Hilpisch | The Python Quants GmbH

DX Analytics (the "dx library" or "dx package") is licensed under the GNU Affero General
Public License version 3 or later (see http://www.gnu.org/licenses/).

DX Analytics comes with no representations or warranties, to the extent
permitted by applicable law.

http://tpq.io | [dx@tpq.io](mailto:team@tpq.io) |
http://twitter.com/dyjh

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

**Quant Platform** | http://pqp.io

**Python for Finance Training** | http://training.tpq.io

**Certificate in Computational Finance** | http://compfinance.tpq.io

**Derivatives Analytics with Python (Wiley Finance)** |
http://dawp.tpq.io

**Python for Finance (2nd ed., O'Reilly)** |
http://py4fi.tpq.io